In [1]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1, 1))
display.start()

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import gym
import numpy as np
import cv2

In [2]:
from stable_baselines.common.policies import MlpLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines import PPO2
from stable_baselines.common import make_vec_env, set_global_seeds

/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: P

In [3]:
def evaluate_2(model, num_episodes=100):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes
    :return: (float) Mean reward for the given number of episodes
    """
    episode_rewards = []
    obs = env.reset()
    for i in range(num_episodes):
        episode_rewards.append(0.0)
        done = False
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            reward, done, info = reward[0], done[0], info[0]
            # Stats
            episode_rewards[-1] += reward
            if done:
                obs = env.reset()
    # Compute mean reward for the last 100 episodes
    mean_100ep_reward = round(np.mean(episode_rewards), 1)
    print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))
    return mean_100ep_reward

In [4]:
def make_env(env_id, rank, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init


# Stable Baselines provides you with make_vec_env() helper
# which does exactly the previous steps for you:
# env = make_vec_env(env_id, n_envs=num_cpu, seed=0)

In [5]:
env_id = "BipedalWalker-v2"
num_cpu = 10  # Number of processes to use
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [6]:
%%time
model = PPO2(MlpLstmPolicy, env, n_steps=int(2000/num_cpu), verbose=1, nminibatches=10)
model.learn(total_timesteps=50000, log_interval=10)





Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


--------------------------------------
| approxkl           | 0.004696714   |
| clipfrac           | 0.049250007   |
| explained_variance | 0.000751      |
| fps                | 169           |
| n_updates          | 1             |
| policy_entropy     | 5.6778383     |
| policy_loss        | -0.0050944868 |
| serial_timesteps   | 200           |
| time_elapsed       | 3.81e-06      |
| total_timesteps    | 2000          |
| value_loss         | 360.61795     |
--------------------------------------
-------------------------------------
| approxkl           | 0.0046669273 |
| clipfrac           | 0.049124993  |
| explained_variance | 0.00144      |
| fps                | 605          |
| n_updates          | 10           |
| policy_entropy     | 5.6540484    |
| policy_loss        | -0.004897359 |
| serial_timesteps   | 2000

In [9]:
env_id = "BipedalWalker-v2"
num_cpu = 20  # Number of processes to use
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_cpu)])

In [10]:
%%time
model = PPO2(MlpLstmPolicy, env, n_steps=int(2000/num_cpu), verbose=1, nminibatches=20)
model.learn(total_timesteps=500000, log_interval=10)

--------------------------------------
| approxkl           | 0.0056247097  |
| clipfrac           | 0.065874994   |
| explained_variance | 0.001         |
| fps                | 324           |
| n_updates          | 1             |
| policy_entropy     | 5.681148      |
| policy_loss        | -0.0071662366 |
| serial_timesteps   | 100           |
| time_elapsed       | 2.86e-06      |
| total_timesteps    | 2000          |
| value_loss         | 206.15178     |
--------------------------------------
--------------------------------------
| approxkl           | 0.008286683   |
| clipfrac           | 0.110249996   |
| explained_variance | 0.334         |
| fps                | 694           |
| n_updates          | 10            |
| policy_entropy     | 5.639859      |
| policy_loss        | -0.0015806978 |
| serial_timesteps   | 1000          |
| time_elapsed       | 30.6          |
| total_timesteps    | 20000         |
| value_loss         | 0.14522584    |
-------------------------

KeyboardInterrupt: 

In [11]:
model.save("ppo2_lstm_parallel_test")

In [12]:
env = gym.make('BipedalWalker-v2')
env = DummyVecEnv([lambda: env])
model = PPO2.load("ppo2_lstm_parallel_test", env=env, nminibatches=16)

AssertionError: Error: the environment passed must have the same number of environments as the model was trained on.This is due to the Lstm policy not being capable of changing the number of environments.

In [ ]:
evaluate_2(model, 10)[0]

In [5]:
# multiprocess environment
env = make_vec_env('BipedalWalker-v2', n_envs=16)

model = PPO2(MlpLnLstmPolicy, env, verbose=1, nminibatches=16)
model.learn(total_timesteps=1600*100)
model.save("ppo2_parallel_lstm_ln_16")






Instructions for updating:
Use keras.layers.flatten instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


-----------------------------------
| approxkl           | 0.07994852 |
| clipfrac           | 0.5003662  |
| ep_len_mean        | 76.2       |
| ep_reward_mean     | -113       |
| explained_variance | 0.00366    |
| fps                | 56         |
| n_updates          | 1          |
| policy_entropy     | 5.6791024  |
| policy_loss        | 0.0207684  |
| serial_timesteps   | 128        |
| time_elapsed       | 2.86e-06   |
| total_timesteps    | 2048       |
| value_loss         | 249.15819  |
-----------------------------------
--------------------------------------
| approxkl           | 0.03558476    |
| clipfrac           | 0.3560791     |
| ep_len_mean        | 83.3          |
| ep_reward_mean     | -115          |
| explained_variance | 0.00645       |
| fps                | 379           |
| n_updates          | 2

In [6]:
model.learn(total_timesteps=1600*400)
model.save("ppo2_parallel_lstm_ln_16_500")

-------------------------------------
| approxkl           | 0.050681584  |
| clipfrac           | 0.35949707   |
| ep_len_mean        | 56           |
| ep_reward_mean     | -109         |
| explained_variance | 0.0851       |
| fps                | 380          |
| n_updates          | 1            |
| policy_entropy     | 5.225996     |
| policy_loss        | -0.021137608 |
| serial_timesteps   | 128          |
| time_elapsed       | 4.53e-06     |
| total_timesteps    | 2048         |
| value_loss         | 49.50274     |
-------------------------------------
------------------------------------
| approxkl           | 0.059793115 |
| clipfrac           | 0.42456055  |
| ep_len_mean        | 89.4        |
| ep_reward_mean     | -109        |
| explained_variance | 0.161       |
| fps                | 382         |
| n_updates          | 2           |
| policy_entropy     | 5.2204256   |
| policy_loss        | -0.03716275 |
| serial_timesteps   | 256         |
| time_elapsed       | 

In [7]:
env

In [21]:
evaluate_2(model)

Mean reward: 9.4 Num episodes: 100


9.4